# Unlocking (and Visualizing) the Airwaves

The [Unlocking the Airwaves](https://mith.umd.edu/airwaves) ia a [virtual reunification](https://www.journals.uchicago.edu/doi/10.1086/676489) project that makes a collection of radio recordings from the [National Association of Educational Broadcasters](https://en.wikipedia.org/wiki/National_Association_of_Educational_Broadcasters) held at the University of Maryland available along with archival documents held at the Wisconsin Historical Society. The website is a client side application that usese some datasets live for searching and browsing. One of those datasets is `episodes.json` which contains a list of JSON objects for each of the NAEB episodes.

In [43]:
import requests

episodes = requests.get('https://unlocking.netlify.app/data/episodes.json').json()

Here's what one of the episode objects looks like:

In [44]:
import json

print(json.dumps(episodes[0], indent=2))

{
  "aapbId": "cpb-aacip-500-0k26f92s",
  "naebId": "62-41-1",
  "umdId": "1619",
  "title": "Mexican history",
  "broadcastDate": "1962-08-15",
  "duration": 1756,
  "seriesDescription": "Panel discussion series about Mexico.",
  "description": "Mexican History. Dr. Thomas McGann, Dr. Karl Schmitt, both of University of Texas.",
  "location": "Hornbake Special Collections",
  "temporal": "1961-1970",
  "language": "English",
  "year": "1962",
  "series": {
    "id": "speaking-of-mexico",
    "title": "Speaking of Mexico"
  },
  "subject": [
    {
      "id": "economy-business",
      "name": "Economy & Business"
    },
    {
      "id": "info-orient",
      "name": "Info.-Orient"
    },
    {
      "id": "latin-america",
      "name": "Latin America"
    },
    {
      "id": "mexico-history",
      "name": "Mexico--History"
    },
    {
      "id": "naeb-region-v",
      "name": "NAEB Region V"
    },
    {
      "id": "naeb-region-vi-lower-midwestcentral-south",
      "name": "NAEB R

One thing we can do is look at the social network of creators of these episodes and consider people connected if they've worked together.

In [71]:
import networkx
import itertools

g = networkx.Graph()

for episode in episodes:
    creators = filter(lambda p: p['type'] == 'Person', episode.get('creator', []))
    creators = set(map(lambda p: p['name'], creators))
    
    # get all pairwise combinations
    for p1, p2 in itertools.combinations(creators, 2):
        if p1 == p2:
            continue
        if g.has_edge(p1, p2):
            weight = g.edges[p1, p2]['weight']
        else:
            weight = 0
        g.add_edge(p1, p2, weight=weight + 1)
        
print(len(g))

287


In [105]:
! pip install --upgrade pyvis

  Attempting uninstall: pyvis
    Found existing installation: pyvis 0.1.8.2
    Uninstalling pyvis-0.1.8.2:
      Successfully uninstalled pyvis-0.1.8.2


In [106]:
from pyvis.network import Network
from IPython.core.display import display, HTML

nt = Network('1400px', '1400px', notebook=True,)
nt.from_nx(g)
nt.show('data/uta-network.html')

That's kind of a tangle. I wonder what it would look like if we remove the edges where the weight is equal to 1, so that we can see people who worked together on more than one episode.

In [117]:
# remove any edges where the weight is 1
g2 = g.copy()
for n1, n2, data in g.edges(data=True):
    if data['weight'] == 1:
        g2.remove_edge(n1, n2)

# remove nodes that are no longer connected
rm_nodes = list(filter(lambda n: g2.degree(n) == 0, g2.nodes()))
g2.remove_nodes_from(rm_nodes)

nt = Network('500px', '1000px', notebook=True, heading="Unlocking the Airwaves")
nt.from_nx(g2)
nt.show_buttons()
nt.show('data/uta-network.html')
display(HTML('data/uta-network.html'))

That still looks a bit complicated. How about if we limit to people who have particular roles?